### Coursework 2

In this coursework you will be aiming to complete two classification tasks. 
Both the classification tasks relate to text classification tasks. 

One task is to be solved using Support Vector Machines. The other has to be solved using Boosting.

The specific tasks and the marking for the various tasks are provided in the notebook. Each task is expected to be accompanied by a lab-report. Each task can have a concise lab report that is maximum of one page in an A4 size. You will be expected to submit your Jupyter Notebook and all lab reports as a single zip file. You could have additional functions implemented that you require for carrying out each task.


#### Task 1

In this task, you need to obtain sentiment analysis for the provided dataset. The dataset consists of movie reviews with the sentiments being provided. The sentiments are either positive or negative. You need to train an SVM based classifier to obtain train and check on the sample test dataset provided. The method will be evaluated also against an external test set. Please do not hardcode any dimensions or number of samples while writing the code. It should be possible to automate the testing and hardcoding values does not allow for automated testing. 

You are allowed to use scikit-learn to implement the SVM. However, you are expected to write your own kernels.

You are allowed to use the existing library functions such as scikit-learn or numpy for obtaining the SVM. The main idea is to analyse the dataset using different kind of kernels. You are also supposed to write your own custom text kernels. Refer to the documentation provided [here](https://scikit-learn.org/stable/modules/svm.html) at 1.4.6.2 and an example [here](https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html) for writing your own kernels.

Details regarding the marking have been provided in the coursework specification file. Ensure that the code can be run with different test files. 

#### Process the text and obtain a bag of words-based features 

In [1]:
import numpy as np
import nltk
import pandas as pd

data_cache = {'Train_path':'', 'Test_path':'', 'X_train':None, 'y_train':None, 'X_test': None, 'y_train':None}

def clear_cache():
    data_cache['Test_path'] = ''
    data_cache['Train_path'] = ''
    data_cache['X_train'] = None
    data_cache['y_train'] = None
    data_cache['X_test'] = None
    data_cache['y_test'] = None

def load_data(train_file, test_file):
    # Read the CSV file and extract Bag of Words Features
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    raw_train_x = list(train_df.review)
    train_y = list(map((lambda x : 0 if x == 'negative' else 1), train_df.sentiment))
    
    raw_test_x = list(test_df.review)
    test_y = list(map((lambda x : 0 if x == 'negative' else 1), test_df.sentiment))
    
    return raw_train_x, np.array(train_y), raw_test_x, np.array(test_y)

def extract_bag_of_words_train_test(train_file, test_file):
    if(data_cache['X_train'] is not None 
        and data_cache['y_train'] is not None
        and data_cache['X_test']is not None
        and data_cache['y_test'] is not None
        and data_cache['Train_path'] == train_file
        and data_cache['Test_path'] == test_file):
        print('Getting from cache')
        return (data_cache['X_train'], data_cache['y_train'], data_cache['X_test'], data_cache['y_test'])
    
    raw_train_x, y_train, raw_test_x, y_test = load_data(train_file, test_file)
    
    X_train = get_embeddings(raw_train_x)
    X_test  = get_embeddings(raw_test_x)
    data_cache['Test_path'] = test_file
    data_cache['Train_path'] = train_file
    data_cache['X_train'] = X_train
    data_cache['y_train'] = y_train
    data_cache['X_test'] = X_test
    data_cache['y_test'] = y_test
    return (X_train, y_train, X_test, y_test)

def get_embeddings(reviews):
    from sentence_transformers import SentenceTransformer
    embedding_length = 128
    embeddings = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
    model.max_seq_length = embedding_length
    
    for review in reviews:
        if len(review.split(' ')) <= embedding_length:
            embeddings.append(model.encode(review))
        else:
            ## If the review is longer than the maximum supported sequence length then embed in chunks and average
            ## chunks overlap by 15 words to preserve some semantic continuity between chunks
            split_review = review.split(' ')
            split_review = [' '.join(split_review[i:i+embedding_length+15]) for i in range(0,len(split_review),embedding_length-15)]
            split_embeddings = []
            split_lens = []
            for split in split_review:
                split_lens.append(len(split))
                embedding = model.encode(split)
                split_embeddings.append(embedding)
            embeddings.append(np.average(np.array(split_embeddings), axis = 0, weights = split_lens))
    return np.array(embeddings)

In [2]:
import numpy as np
from sklearn import svm
import itertools

class SVMClassifier:
    def __init__(self):
        
        #implement initialisation
        #self.some_paramter=1
        self.clf = svm.SVC(decision_function_shape='ovr', C=0.9,  gamma=2.2857142857142856)
        self.clf.kernel = 'rbf'
        
    # define your own kernel here
    # Refer to the documentation here: https://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html
    def fit(self, X, y):
        # training of the SVM
        # Ensure you call your own defined kernel here
        # defined below as own method to permit hyperparam tuning via builder method
        self.clf.fit(X, y)
               
        return
    
    def predict(self, X):
        # prediction routine for the SVM
        predictions = self.clf.predict(X)
        
        return predictions
    
def linear_kernel_builder(M):
    # effectively multiply each feature by some weights and dot product with expected output
    def linear_kernel(X, Y): 
        weighted_features = (X * M.T)
        out = weighted_features @ Y.T
        return out
    return linear_kernel
    
def cross_validation(model, params, X_train, Y_train, kfolds):
    from sklearn.model_selection import KFold
    from sklearn.metrics import accuracy_score
    k_folder = KFold(n_splits=kfolds, shuffle=True)

    def test_svm(train_x, train_y, test_x, test_y):
        model.fit(train_x, train_y)
        Y_Pred = model.predict(test_x)
        return accuracy_score(test_y, Y_Pred)

    accuracy = []
    for train_index, val_index in k_folder.split(X_train):
        train_x = X_train[train_index, :]
        train_y = Y_train[train_index]
        val_x = X_train[val_index, :]
        val_y = Y_train[val_index]

        acc = test_svm(train_x, train_y, val_x, val_y)
        accuracy.append(acc)

    return (params, np.mean(accuracy))
    
# Find best hyper-params via grid search with k-fold cross-validation
def hyperparam_tuning(model_builder, search_space, X_train, Y_train, kfolds=5):
    import multiprocessing as mp
    
    cross_val_history = [] # (params, mean_acc)
    count = 0
    for params in itertools.product(*search_space):

        cross_val_history.append(cross_validation(model_builder(params), params, X_train, Y_train, kfolds))
    
    best_params = max(cross_val_history, key=lambda run: run[1])
    
    return best_params, cross_val_history


### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [3]:
def test_func_svm(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score  
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    sc = SVMClassifier()
    sc.fit(X_train, Y_train)
    Y_Pred = sc.predict(X_test)
    acc = accuracy_score(Y_test, Y_Pred)
    print("Accuracy:",acc)
    return acc

In [4]:
acc = test_func_svm("movie_review_train.csv", "movie_review_test.csv")

Accuracy: 0.844


### Task 2

In this task you need to implement a boosting based classifier that can be used to classify the images. 

Details regarding the marking for the coursework are provided in the coursework specification file. Please ensure that your code will work with a different test file than the one provided with the coursework.

Note that the boosting classifier you implement can include decision trees from scikit-learn or your own decision trees. Use the same sentiment analysis dataset for evaluation.

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
import math
import random

class BoostingClassifier:
    # You need to implement this classifier. 
    def __init__(self, num_classifiers = 50, max_depth = 3):
        import numpy as np
        #implement initialisation
        self.num_classifiers = num_classifiers
        self.max_depth = max_depth
        self.classifiers = []
        self.alphas = np.ones(self.num_classifiers)
        self.alphas /= self.num_classifiers
        
    def fit(self, X,y):
        signed_y = [1 if x == 1 else -1 for x in y]
        num_samples = X.shape[0]
        w = np.random.rand(num_samples)
        w /= num_samples
        alph = np.ones(self.num_classifiers)
        for c in range(self.num_classifiers):
            clf = tree.DecisionTreeClassifier(max_depth = self.max_depth)#splitter='random')
            self.classifiers.append(clf)
            clf.fit(X, signed_y, sample_weight = w)
            
            #get predictions and calculate error rate
            predictions = clf.predict(X)     
            error_rate = np.count_nonzero(predictions != signed_y) / num_samples
            #set classifier weight
            alpha = 0.5 * math.log((1-error_rate)/error_rate) if error_rate > 0 else 1
            self.alphas[c]  = alpha
            #update sample weights
            
            new_w = np.copy(w)
            for i in range(num_samples):
                w[i] = w[i] * math.exp(-1 * alpha * signed_y[i] * predictions[i])
                
            w /= np.sum(new_w)
        
        #implement training of the boosting classifier
        return self
    
    def get_params(self, deep= False):
        return {'num_classifiers':self.num_classifiers, 'max_depth':self.max_depth}
    
    def predict(self, X):
        # implement prediction of the boosting classifier
        h = []
        accum = []
        for x in X:
            cumsum = 0
            for c in range(self.num_classifiers):
                cumsum += self.alphas[c] * self.classifiers[c].predict(np.array(x).reshape(1,-1))
                accum.append(cumsum)
            h.append(np.sign(cumsum))
        return [0 if x < 0 else 1 for x in h]
    



In [6]:
from sklearn.metrics import accuracy_score  
def crossval_boost(model, x, y, kfolds = 5):
    from sklearn.model_selection import KFold

    kfold_accuracys = []

    kf = KFold(n_splits = kfolds)
    train_x = kf.split(x,y)


    for train_index, val_index in kf.split(x):
        train_x = x[train_index]
        train_y = y[train_index]
        val_x = x[val_index]
        val_y = y[val_index]

        bc = BoostingClassifier()
        bc.fit(train_x, train_y)
        Y_Pred = bc.predict(val_x)
        acc = accuracy_score(val_y, Y_Pred)

        kfold_accuracys.append(acc)

    return np.mean(kfold_accuracys)

## Hyperparam tuning for gradboost
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from scipy.stats import uniform
def grid_search(params):
    X_train, y_train, X_test, Y_test = extract_bag_of_words_train_test("movie_review_train.csv", "movie_review_test.csv")
    clfs = []
    best_acc = 0

    for num in params['num_classifiers']:
        for max_depth in params['max_depth']:
            clfs.append(BoostingClassifier(num, max_depth))
    for clf in clfs:
        accuracy = crossval_boost(clf, X_train, np.array(y_train), 5)
        print('Acc:' , accuracy, 'Params:', clf.num_classifiers, clf.max_depth)
        if accuracy > best_acc:
            best_clf = clf
            best_acc = accuracy

    print("Best Params:", best_clf.num_classifiers, best_clf.max_depth)
    return best_clf

grid = {'num_classifiers': [50, 100, 200, 500], 'max_depth': [2, 5, 10, 25]}
#best_clf = grid_search(grid)



### Test function that will be called to evaluate your code. Separate test dataset will be provided

Do not modify the code below. Please write your code above such that it can be evaluated by the function below. You can modify your code above such that you obtain the best performance through this function. 

In [7]:
def test_func_boosting(dataset_train, dataset_test):
    from sklearn.metrics import accuracy_score    
    (X_train, Y_train, X_test, Y_test) = extract_bag_of_words_train_test(dataset_train, dataset_test)
    bc = BoostingClassifier()
    bc.fit(X_train, Y_train)
    Y_Pred = bc.predict(X_test)    
    acc = accuracy_score(Y_test, Y_Pred)
    return acc

In [9]:
acc = test_func_boosting("movie_review_train.csv", "movie_review_test.csv")
print(acc)

0.768
